# Importar Bibliotecas

In [97]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import psycopg2
from psycopg2 import OperationalError
import os
from dotenv import load_dotenv

In [98]:
# Configurações iniciais
load_dotenv()
np.random.seed(0)  # Para garantir a reprodutibilidade

# Definindo parâmetros para a geração de dados

In [99]:
# Configurações iniciais
np.random.seed(0)  # Para garantir a reprodutibilidade

# Parâmetros de geração de dados
tipos_cerveja = ["Ale", "Wheat Beer", "Pilsen", "Lager", "IPA", "Stout", "Porter"]
quantidades = {
    "Ale": (3000, 7000),
    "Wheat Beer": (2000, 6000),
    "Pilsen": (2500, 7500),
    "Lager": (3000, 8000),
    "IPA": (2000, 5000),
    "Stout": (1500, 4500),
    "Porter": (1000, 4000),
}
custos_por_unidade = {
    "Ale": (0.10, 0.30),
    "Wheat Beer": (0.15, 0.45),
    "Pilsen": (0.10, 0.30),
    "Lager": (0.08, 0.25),
    "IPA": (0.20, 0.50),
    "Stout": (0.25, 0.55),
    "Porter": (0.30, 0.60),
}
precos_minimos = {
    "Ale": 2.9,
    "Wheat Beer": 3.0,
    "Pilsen": 2.2,
    "Lager": 3.1,
    "IPA": 3.2,
    "Stout": 3.5,
    "Porter": 4.0,
}
margens_de_lucro = {
    "Ale": 0.50,
    "Wheat Beer": 0.45,
    "Pilsen": 0.40,
    "Lager": 0.55,
    "IPA": 0.42,
    "Stout": 0.40,
    "Porter": 0.45,
}

# Gerando dados de produção (Lotes)

In [100]:
# Gerando dados de produção (Lotes)
dados_producao = pd.DataFrame({
    "ID_Lote": range(1, 251),
    "Data_Producao": pd.date_range(start="2020-01-01", periods=250, freq='D'),
    "Tipo_Cerveja": np.random.choice(tipos_cerveja, 250),
})

# Calculando a quantidade produzida e custo de produção
dados_producao['Quantidade_Produzida'] = [np.random.randint(*quantidades[tc]) for tc in dados_producao['Tipo_Cerveja']]
dados_producao['Custo_Producao'] = [qtd * np.random.uniform(*custos_por_unidade[tc]) for tc, qtd in zip(dados_producao['Tipo_Cerveja'], dados_producao['Quantidade_Produzida'])]

# Calculando o preço de venda
dados_producao['Preco_Venda'] = [max(custo / qtd * (1 + margens_de_lucro[tc]), precos_minimos[tc]) for tc, qtd, custo in zip(dados_producao['Tipo_Cerveja'], dados_producao['Quantidade_Produzida'], dados_producao['Custo_Producao'])]

# Adicionando outros dados
dados_producao['Lote_Ingredientes'] = np.random.randint(100, 200, 250)
dados_producao['Tempo_Fermentacao'] = np.random.randint(1, 14, 250)
dados_producao['Data_Validade'] = pd.date_range(start="2021-01-01", periods=250, freq='30D')

# Mostrar os dados gerados
print(dados_producao)


     ID_Lote Data_Producao Tipo_Cerveja  Quantidade_Produzida  Custo_Producao  \
0          1    2020-01-01          IPA                  4022     1954.133753   
1          2    2020-01-02        Stout                  1791      455.244473   
2          3    2020-01-03          Ale                  4750      981.792398   
3          4    2020-01-04        Lager                  6781      893.593028   
4          5    2020-01-05        Lager                  5775     1328.747803   
..       ...           ...          ...                   ...             ...   
245      246    2020-09-02        Lager                  6516     1619.187430   
246      247    2020-09-03       Pilsen                  7199     1271.491543   
247      248    2020-09-04        Lager                  5671     1231.365696   
248      249    2020-09-05       Pilsen                  4887      939.288075   
249      250    2020-09-06          IPA                  3868     1733.046818   

     Preco_Venda  Lote_Ingr

# Dados de vendas (2000 entradas ao longo de 3 anos)

In [101]:
dados_vendas = pd.DataFrame({
    "ID_Venda": range(1, 4341),
    "Data_Venda": pd.date_range(start="2020-01-01", periods=4340, freq='D'),
    "ID_Cliente": np.random.randint(1, 500, 4340),
    "ID_Lote": np.random.choice(dados_producao["ID_Lote"], 4340),
    "Quantidade_Vendida": np.random.randint(1, 100, 4340),
    "Preco_Venda": np.random.uniform(5, 50, 4340),
    "Canal_Venda": np.random.choice(["Loja Física", "Online", "Distribuidor"], 4340)
})

# Dados de devolução (13% das vendas)

In [102]:
# Gerando dados de devolução
quantidade_devolucoes = int(0.13 * len(dados_vendas))
dados_devolucao = dados_vendas.sample(n=quantidade_devolucoes).copy()

# Renomear coluna e adicionar colunas necessárias
dados_devolucao = dados_devolucao.rename(columns={"Quantidade_Vendida": "Quantidade_Devolvida"})
dados_devolucao["ID_Devolucao"] = range(1, quantidade_devolucoes + 1)
dados_devolucao["Motivo_Devolucao"] = np.random.choice(["Defeito", "Insatisfação", "Outro"], quantidade_devolucoes)
dados_devolucao["Estado_Produto"] = np.random.choice(["Bom", "Danificado", "Expirado"], quantidade_devolucoes)
dados_devolucao["Reembolso_Emitido"] = np.random.uniform(5, 50, quantidade_devolucoes)

# Removendo colunas que não existem na tabela Devolucoes do banco de dados
colunas_a_remover = [coluna for coluna in dados_devolucao.columns if coluna not in ['ID_Devolucao', 'ID_Venda', 'Quantidade_Devolvida', 'Motivo_Devolucao', 'Estado_Produto', 'Reembolso_Emitido']]
dados_devolucao = dados_devolucao.drop(columns=colunas_a_remover)

# Ajustando a ordem das colunas para corresponder à estrutura da tabela no banco de dados
dados_devolucao = dados_devolucao[['ID_Devolucao', 'ID_Venda', 'Quantidade_Devolvida', 'Motivo_Devolucao', 'Estado_Produto', 'Reembolso_Emitido']]


# Dados de feedback (gerados aleatoriamente)

In [103]:
# Gerando dados de feedback
quantidade_feedbacks = random.randint(100, 300)
dados_feedback = pd.DataFrame({
    "ID_Feedback": range(1, quantidade_feedbacks + 1),
    "Data_Feedback": pd.date_range(start="2020-01-01", end="2020-12-31", periods=quantidade_feedbacks),
    "ID_Venda": np.random.choice(dados_vendas["ID_Venda"], quantidade_feedbacks),
    "Tipo_Feedback": np.random.choice(["Produto", "Serviço", "Entrega"], quantidade_feedbacks),
    "Descricao_Feedback": np.random.choice(["Excelente", "Bom", "Médio", "Ruim", "Péssimo"], quantidade_feedbacks),
    "Resposta_Cervejaria": np.random.choice(["Resolvido", "Pendente", "Ignorado"], quantidade_feedbacks)
})
print(dados_devolucao)

      ID_Devolucao  ID_Venda  Quantidade_Devolvida Motivo_Devolucao  \
1697             1      1698                    20          Defeito   
964              2       965                    12     Insatisfação   
870              3       871                    47     Insatisfação   
3086             4      3087                     6          Defeito   
2868             5      2869                    27            Outro   
...            ...       ...                   ...              ...   
1843           560      1844                    43            Outro   
3456           561      3457                     9     Insatisfação   
1388           562      1389                    88          Defeito   
2466           563      2467                    72     Insatisfação   
538            564       539                    56          Defeito   

     Estado_Produto  Reembolso_Emitido  
1697            Bom          11.131967  
964      Danificado          17.818737  
870      Danificado     

In [104]:
print(dados_producao)

     ID_Lote Data_Producao Tipo_Cerveja  Quantidade_Produzida  Custo_Producao  \
0          1    2020-01-01          IPA                  4022     1954.133753   
1          2    2020-01-02        Stout                  1791      455.244473   
2          3    2020-01-03          Ale                  4750      981.792398   
3          4    2020-01-04        Lager                  6781      893.593028   
4          5    2020-01-05        Lager                  5775     1328.747803   
..       ...           ...          ...                   ...             ...   
245      246    2020-09-02        Lager                  6516     1619.187430   
246      247    2020-09-03       Pilsen                  7199     1271.491543   
247      248    2020-09-04        Lager                  5671     1231.365696   
248      249    2020-09-05       Pilsen                  4887      939.288075   
249      250    2020-09-06          IPA                  3868     1733.046818   

     Preco_Venda  Lote_Ingr

In [105]:
# Verificando se a coluna 'Preco_Venda' existe em 'dados_producao' antes de tentar removê-la
if 'Preco_Venda' in dados_producao.columns:
    dados_producao = dados_producao.drop(columns=['Preco_Venda'])

# Removendo as colunas desnecessárias de 'dados_devolucao' e adicionando 'Reembolso_Emitido'
if 'Data_Venda' in dados_devolucao.columns:
    dados_devolucao = dados_devolucao.drop(columns=['Data_Venda', 'ID_Cliente', 'Preco_Venda', 'Canal_Venda'])

dados_devolucao['Reembolso_Emitido'] = dados_devolucao['Motivo_Devolucao'].apply(lambda x: 1.0 if x in ['Defeito', 'Insatisfação'] else 0.0)


## Inserindo dados faltantes no dataframe de feedback



In [106]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Obtém as informações de conexão do arquivo .env
host = os.getenv('DB_HOST', 'localhost')
porta = os.getenv('DB_PORT', '5432')
dbname = os.getenv('DB_NAME', 'postgres')
usuario = os.getenv('DB_USER', 'postgres')
senha = os.getenv('DB_PASSWORD', 'default_password')

def create_connection(host_name, port, database_name, user_name, user_password):
    connection = None
    try:
        connection = psycopg2.connect(
            host=host_name,
            port=port,
            dbname=database_name,
            user=user_name,
            password=user_password
        )
        print("Conexão com o PostgreSQL estabelecida")
    except OperationalError as e:
        print(f"Ocorreu um erro ao conectar ao PostgreSQL: {e}")
    return connection

# Estabelecendo a conexão
conn = create_connection(host, porta, dbname, usuario, senha)

# Lembre-se de fechar a conexão quando terminar
# conn.close()

Conexão com o PostgreSQL estabelecida


In [107]:
def create_tables(connection):
    cursor = connection.cursor()

    # Criação da tabela de produção
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Producao (
        ID_Lote INTEGER PRIMARY KEY,
        Tipo_Cerveja TEXT,
        Data_Producao DATE,
        Quantidade_Produzida INTEGER,
        Custo_Producao REAL,
        Lote_Ingredientes TEXT,
        Tempo_Fermentacao INTEGER,
        Data_Validade DATE
    )
    """)

    # Criação da tabela de vendas
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Vendas (
        ID_Venda INTEGER PRIMARY KEY,
        ID_Lote INTEGER,
        Data_Venda DATE,
        Quantidade_Vendida INTEGER,
        Preco_Venda REAL,
        ID_Cliente INTEGER,
        Canal_Venda TEXT,
        FOREIGN KEY (ID_Lote) REFERENCES Producao (ID_Lote)
    )
    """)

    # Criação da tabela de devoluções
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Devolucoes (
        ID_Devolucao INTEGER PRIMARY KEY,
        ID_Venda INTEGER,
        Quantidade_Devolvida INTEGER,
        Motivo_Devolucao TEXT,
        Estado_Produto TEXT,
        Reembolso_Emitido REAL,
        FOREIGN KEY (ID_Venda) REFERENCES Vendas (ID_Venda)
    )
    """)

    # Criação da tabela de feedback
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Feedback (
        ID_Feedback INTEGER PRIMARY KEY,
        Data_Feedback DATE,
        ID_Cliente INTEGER,
        Tipo_Feedback TEXT,
        Descricao_Feedback TEXT,
        Resposta_Cervejaria TEXT,
        ID_Venda INTEGER,
        FOREIGN KEY (ID_Venda) REFERENCES Vendas (ID_Venda)
    )
    """)

    connection.commit()
    cursor.close()


In [108]:
def insert_data(df, table_name, connection):
    cursor = connection.cursor()
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    try:
        for row in df.itertuples(index=False, name=None):
            cursor.execute(query, row)
        connection.commit()
    except psycopg2.DatabaseError as e:
        print(f"Erro ao inserir dados na tabela {table_name}: {e}")
        connection.rollback()  # Reverter a transação em caso de erro
    finally:
        cursor.close()

# Inserir dados
insert_data(dados_producao, 'Producao', conn)
insert_data(dados_vendas, 'Vendas', conn)
insert_data(dados_devolucao, 'Devolucoes', conn)  # Inclua todas as colunas do DataFrame
insert_data(dados_feedback, 'Feedback', conn)

In [109]:
conn.close()